In [1]:
from peft import AutoPeftModelForSequenceClassification
from transformers import AutoTokenizer, BitsAndBytesConfig
import torch
import evaluate
from datasets import load_dataset
from torch.nn import CrossEntropyLoss

/fs01/home/dadsetan/condaenvs/lora-finetuning-mia/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/fs01/home/dadsetan/condaenvs/lora-finetuning-mia/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
bnb_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.bfloat16
)

In [3]:
id2label = {0: "negative", 1: "neutral", 2: "positive"}
label2id = {"negative": 0, "neutral": 1, "positive": 2}

In [4]:
device = "cuda"
model = AutoPeftModelForSequenceClassification.from_pretrained(
    "/h/dadsetan/workspace/lora-finetuning-mia/local/tweet_shield_dp_trained_model/checkpoint-12267",
    num_labels=3,
    quantization_config=bnb_config
)
model = model.to(device)
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at /model-weights/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): LlamaForSequenceClassification(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
          

In [5]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): LlamaForSequenceClassification(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
          

In [7]:
score_weights = torch.load("/h/dadsetan/workspace/lora-finetuning-mia/local/tweet_shield_dp_trained_model/checkpoint-12267/score.original_module.pt", map_location="cpu")
model.score.original_module.load_state_dict(score_weights)

<All keys matched successfully>

In [11]:
tokenizer = AutoTokenizer.from_pretrained("/h/dadsetan/workspace/lora-finetuning-mia/local/tweet_shield_dp_trained_model/checkpoint-12267")

In [12]:
model.device

device(type='cuda', index=0)

In [13]:
orig_dataset = load_dataset("parquet", data_files="./data_balanced.parquet")
# orig_dataset = load_dataset("parquet", data_files="./anonymized.parquet")

dataset = orig_dataset.rename_column("Lable", "label").rename_column("Text", "text")
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)

def preprocess_fn(examples):
  return tokenizer(examples["text"], truncation=True)

toked = dataset.map(preprocess_fn, batched=True)

def change_labels(example):
  label = example["label"]
  n_label = 0 if label == "negative" else 1 if label == "neutral" else 2
  example["label"] = n_label
  return example

toked = toked.map(change_labels)

accuracy = evaluate.load("accuracy")



Map:   0%|          | 0/8183 [00:00<?, ? examples/s]

Map:   0%|          | 0/910 [00:00<?, ? examples/s]

Map:   0%|          | 0/8183 [00:00<?, ? examples/s]

Map:   0%|          | 0/910 [00:00<?, ? examples/s]

In [14]:
anonymized_dataset = load_dataset("parquet", data_files="./anonymized.parquet")

anonymized_dataset = anonymized_dataset.rename_column("Lable", "label").rename_column("Text", "text")
anonymized_dataset = anonymized_dataset["train"].train_test_split(test_size=0.1, seed=42)

def preprocess_fn(examples):
  return tokenizer(examples["text"], truncation=True)

anonymized_toked = anonymized_dataset.map(preprocess_fn, batched=True)

def change_labels(example):
  label = example["label"]
  n_label = 0 if label == "negative" else 1 if label == "neutral" else 2
  example["label"] = n_label
  return example

anonymized_toked = anonymized_toked.map(change_labels)

Map:   0%|          | 0/8183 [00:00<?, ? examples/s]

Map:   0%|          | 0/910 [00:00<?, ? examples/s]

Map:   0%|          | 0/8183 [00:00<?, ? examples/s]

Map:   0%|          | 0/910 [00:00<?, ? examples/s]

In [15]:
loss_fn = CrossEntropyLoss()
with torch.no_grad():
    text = '"hubby and i enjoyed playing golf for only $MONEY a person. new golf course having what they call DATE madness. cool for us!! "'
    logits = model(**tokenizer(text, return_tensors="pt").to(device))["logits"]
    smax = torch.softmax(logits, dim=-1)
    loss = -torch.log(smax[0,2])
    print(logits)
    print(smax)
    print(loss)
    

tensor([[ 0.3142, -1.5439,  3.9551]], device='cuda:0', dtype=torch.float16)
tensor([[0.0255, 0.0040, 0.9707]], device='cuda:0', dtype=torch.float16)
tensor(0.0297, device='cuda:0', dtype=torch.float16)


In [16]:
train_losses = []
for i,data in enumerate(toked["train"]):
    with torch.no_grad():
        logits = model(input_ids=torch.tensor([data["input_ids"]]).to(device))["logits"]
        train_losses.append(-torch.log(torch.softmax(logits,dim=-1)[0,data["label"]]))
    if i > 5000:
        break

In [17]:
train_losses = torch.stack(train_losses)

In [18]:
train_losses.mean()

tensor(inf, device='cuda:0', dtype=torch.float16)

In [19]:
test_losses = []
for i,data in enumerate(toked["test"]):
    with torch.no_grad():
        logits = model(input_ids=torch.tensor([data["input_ids"]]).to(device))["logits"]
        test_losses.append(-torch.log(torch.softmax(logits,dim=-1)[0,data["label"]]))
    if i > 5000:
        break
test_losses = torch.stack(test_losses)
test_losses.mean()

tensor(0.7041, device='cuda:0', dtype=torch.float16)

In [20]:
print(train_losses.shape)
print(test_losses.shape)
train_sorted, indices = torch.sort(train_losses)

def TPR_FPR(index):
    FPR = index/train_sorted.shape[0]
    true_postive = (test_losses >= train_sorted[-index]).sum()
    TPR = true_postive/test_losses.shape[0]
    return TPR.item(), FPR

TPR_FPR_list = []

for i in range(1,train_sorted.shape[0]):
    TPR_FPR_list.append(TPR_FPR(i))
    print(TPR_FPR(i))

torch.Size([5002])
torch.Size([910])
(0.0, 0.00019992003198720512)
(0.0, 0.00039984006397441024)
(0.0, 0.0005997600959616154)
(0.0, 0.0007996801279488205)
(0.0, 0.0009996001599360256)
(0.0, 0.0011995201919232307)
(0.0, 0.0013994402239104358)
(0.0, 0.001599360255897641)
(0.001098901149816811, 0.001799280287884846)
(0.001098901149816811, 0.001999200319872051)
(0.001098901149816811, 0.0021991203518592563)
(0.002197802299633622, 0.0023990403838464614)
(0.002197802299633622, 0.0025989604158336665)
(0.002197802299633622, 0.0027988804478208716)
(0.002197802299633622, 0.0029988004798080768)
(0.003296703565865755, 0.003198720511795282)
(0.003296703565865755, 0.003398640543782487)
(0.004395604599267244, 0.003598560575769692)
(0.004395604599267244, 0.0037984806077568972)
(0.004395604599267244, 0.003998400639744102)
(0.004395604599267244, 0.004198320671731308)
(0.004395604599267244, 0.004398240703718513)
(0.004395604599267244, 0.004598160735705718)
(0.004395604599267244, 0.004798080767692923)
(0.0

In [21]:
for i,data in enumerate(anonymized_toked["train"]):
    # print(data.keys())
    print("="*40)
    print(f"data point {i}. Label: {data['label']}")
    print("Text: ↓")
    print(data["text"])
    # with torch.no_grad():
    #     logits = model(**tokenizer(data["text"], return_tensors="pt").to(device))["logits"] 
    #     print(logits)
    # print(data["text"])
    # print(data["input_ids"])
    if i > 50:
        break

data point 0. Label: 2
Text: ↓
"helloo kitty hang in there kitty!! "
data point 1. Label: 2
Text: ↓
"iron man1176 laugh out loud you're welcome. sure. i'll send you that email in the morning too before i get busy "
data point 2. Label: 0
Text: ↓
"i had 2 stop smoking hydro cuz im going to the army "
data point 3. Label: 1
Text: ↓
united is it possible to redeem miles for one part of a round trip itinerary without having to book two separate reservations?
data point 4. Label: 1
Text: ↓
jet blue are we scheduled to depart as scheduled?
data point 5. Label: 0
Text: ↓
"i have no socks.!!1.. "
data point 6. Label: 1
Text: ↓
united is hiring  flight dispatcher  chicago, il (network operations centerl)
data point 7. Label: 0
Text: ↓
"mayawati doesn't like to be 'jhappied' or 'pappied' "
data point 8. Label: 1
Text: ↓
american air  american view the view of oregon from seat 31f, flight aa1469 2/22/15
data point 9. Label: 2
Text: ↓
"nfarooq yup! that's one of my favourite songs(and my ringtone)

In [22]:
for i,data in enumerate(toked["train"]):
    # print(data.keys())
    print(f"data point {i}. Label: {data['label']}")
    print("Text: ↓")
    print(data["text"])
    # with torch.no_grad():
    #     logits = model(**tokenizer(data["text"], return_tensors="pt").to(device))["logits"] 
    #     print(logits)
    # print(data["text"])
    # print(data["input_ids"])
    if i > 50:
        break

data point 0. Label: 2
Text: ↓
"helloo kitty hang in there kitty!! "
data point 1. Label: 2
Text: ↓
"iron man1176 laugh out loud you're welcome. sure. i'll send you that email in the morning too before i get busy "
data point 2. Label: 0
Text: ↓
"i had 2 stop smoking hydro cuz im going to the army "
data point 3. Label: 1
Text: ↓
united is it possible to redeem miles for one part of a round trip itinerary without having to book two separate reservations?
data point 4. Label: 1
Text: ↓
jet blue are we scheduled to depart as scheduled?
data point 5. Label: 0
Text: ↓
"i have no socks.!!1.. "
data point 6. Label: 1
Text: ↓
united is hiring  flight dispatcher  chicago, il (network operations centerl)
data point 7. Label: 0
Text: ↓
"mayawati doesn't like to be 'jhappied' or 'pappied' "
data point 8. Label: 1
Text: ↓
american air  american view the view of oregon from seat 31f, flight aa1469 2/22/15
data point 9. Label: 2
Text: ↓
"nfarooq yup! that's one of my favourite songs(and my ringtone)

In [23]:
for i,data in enumerate(toked["test"]):
    # print(data.keys())
    print("="*40)
    print(f"data point {i}. Label: {data['label']}")
    print("Text: ↓")
    print(data["text"])
    # with torch.no_grad():
    #     logits = model(**tokenizer(data["text"], return_tensors="pt").to(device))["logits"] 
    #     print(logits)
    # print(data["text"])
    # print(data["input_ids"])
    if i > 50:
        break

data point 0. Label: 1
Text: ↓
american air if i've booked an alcoholics anonymous award on usair metal, and space opens up on alcoholics anonymous metal, can i change to the alcoholics anonymous flight without fees?
data point 1. Label: 1
Text: ↓
jet blue all up to date. can't update via google play.
data point 2. Label: 0
Text: ↓
"wisdom teeth "
data point 3. Label: 0
Text: ↓
"ugh! the buell has an exhaust leak. "
data point 4. Label: 1
Text: ↓
virgin america do you miss me? don't worry we'll be together very soon.
data point 5. Label: 2
Text: ↓
"here we go. bring on the questions "
data point 6. Label: 1
Text: ↓
southwest air can you follow and we'll proceed?
data point 7. Label: 0
Text: ↓
"i this i have bronchitis comming every time i breath out a buggie tweets three times .."
data point 8. Label: 0
Text: ↓
"en c lasee "
data point 9. Label: 1
Text: ↓
jet blue do you think snow in boston on 2/24 will effect my flight?
data point 10. Label: 2
Text: ↓
"found new band..parachute! haha

In [ ]:
toked["test"][18]

In [ ]:
(torch.tensor(toked["train"]["label"]) == 2).sum()